# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,8.23,97,96,1.79,NZ,1726432531
1,1,askern,53.6164,-1.1524,13.82,88,63,3.09,GB,1726432532
2,2,la passe,45.5549,-0.8967,14.90,51,68,4.50,FR,1726432533
3,3,puerto ayora,-0.7393,-90.3518,22.53,93,76,3.58,EC,1726432534
4,4,ilulissat,69.2167,-51.1000,-0.29,75,99,1.61,GL,1726432535


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,  
    size='Humidity',
    color='City',  # Color points by city name to assign a unique color for each city
    cmap='Category20',  # Use a categorical colormap to assign different colors
    alpha=0.6, 
    hover_cols=['City', 'Humidity', 'Max Temp', 'Wind Speed'],  # Show extra info on hover
    tiles='OSM',
    width=1000,  # Set the width of the plot
    height=600  # Set the height of the plot 
)

# Display the map
humidity_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
# Criterias = max temp lower than 32 degrees but highr than 24, humidity lower than 85 degrees

filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 24) &  # Max temp higher than 24 degrees
    (city_data_df['Max Temp'] < 32) &  # Max temp lower than 32 degrees
    (city_data_df['Humidity'] < 85) &   # Humidity lower than 85%
    (city_data_df['Wind Speed'] < 4)    # Wind speed lower than 4
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,albany,42.6001,-73.9662,28.84,56,1,1.49,US,1726432552
21,21,bethel,41.3712,-73.4140,26.30,59,20,3.09,US,1726432554
33,33,hawaiian paradise park,19.5933,-154.9731,28.26,75,20,1.54,US,1726432568
37,37,nova sintra,14.8667,-24.7167,26.40,76,91,3.41,CV,1726432573
39,39,san patricio,28.0170,-97.5169,30.97,82,100,2.67,US,1726432575
47,47,uturoa,-16.7333,-151.4333,25.65,80,7,3.28,PF,1726432584
64,64,jamestown,42.0970,-79.2353,27.79,35,20,3.58,US,1726432603
70,70,cambria,35.5641,-121.0808,25.69,83,34,2.24,US,1726432611
75,75,mangrol,21.1167,70.1167,26.48,78,9,2.96,IN,1726432617
80,80,niafunke,15.9322,-3.9906,28.86,62,99,1.24,ML,1726432622


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Wind Speed']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Wind Speed,Hotel Name
19,albany,US,42.6001,-73.9662,56,1.49,
21,bethel,US,41.3712,-73.4140,59,3.09,
33,hawaiian paradise park,US,19.5933,-154.9731,75,1.54,
37,nova sintra,CV,14.8667,-24.7167,76,3.41,
39,san patricio,US,28.0170,-97.5169,82,2.67,
47,uturoa,PF,-16.7333,-151.4333,80,3.28,
64,jamestown,US,42.0970,-79.2353,35,3.58,
70,cambria,US,35.5641,-121.0808,83,2.24,
75,mangrol,IN,21.1167,70.1167,78,2.96,
80,niafunke,ML,15.9322,-3.9906,62,1.24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit results to 1 
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
hawaiian paradise park - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
san patricio - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
jamestown - nearest hotel: DoubleTree Jamestown
cambria - nearest hotel: Olallieberry Inn
mangrol - nearest hotel: No hotel found
niafunke - nearest hotel: No hotel found
tofol - nearest hotel: Nautilus Resort
singaraja - nearest hotel: Hotel Sentral
jesus de otoro - nearest hotel: Hotel Tosta
biswan - nearest hotel: No hotel found
gwadar - nearest hotel: Sadaf Resorts
manokwari - nearest hotel: Hotel Arfak
harnai - nearest hotel: No hotel found
mulayjah - nearest hotel: No hotel found
kasongo-lunda - nearest hotel: No hotel found
alofi - nearest hotel: Matavai Resort
nanlong - nearest hotel: No hotel found
numan - nearest hotel: No hotel found
galle - nearest hotel: Jungle Hostel Villa
ain suk

,City,Country,Lat,Lng,Humidity,Wind Speed,Hotel Name
19,albany,US,42.6001,-73.9662,56,1.49,No hotel found
21,bethel,US,41.3712,-73.4140,59,3.09,Hampton Inn Danbury
33,hawaiian paradise park,US,19.5933,-154.9731,75,1.54,No hotel found
37,nova sintra,CV,14.8667,-24.7167,76,3.41,Residência Ka Dencho
39,san patricio,US,28.0170,-97.5169,82,2.67,No hotel found
...,...,...,...,...,...,...,...
547,bowie,US,33.4501,-94.4335,71,2.84,Holiday Inn Express & Suites New Boston
551,mermoz boabab,SN,14.7065,-17.4758,83,1.03,La maison blanche
553,ciudad lazaro cardenas,MX,17.9583,-102.2000,73,3.37,Hotel Sol del Pacífico
554,shakawe,BW,-18.3667,21.8500,20,2.74,EdenWood guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_with_hotels = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,  
    size='Humidity',  # Size points based on humidity
    color='City',  # Differentiates cities by color
    cmap='Category20',  # Use a categorical colormap
    alpha=0.6,  # Transparency
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity', 'Max Temp', 'Wind Speed'],  # Add hotel and country to hover
    tiles='OSM',  # OpenStreetMap as the base layer
    width=1000,  # Adjust width
    height=800   # Adjust height
)

# Display the map
map_with_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name,Wind Speed)